In [1]:
def add(x, y):
    return x + y

def add_wrapper(*args, **kwargs):
    print('Wrapping')
    return add(*args, **kwargs)

In [4]:
result = add_wrapper(5, 3)

Wrapping


In [6]:
result

8

In [8]:
result2 = add_wrapper(y=20, x=12)

Wrapping


In [10]:
result2

32

# Simple Decorator

In [41]:
def add(x, y):
    """Adds x and y"""
    return x + y

def sub(x, y):
    """Substracts x and y"""
    return x - y

def mul(x, y):
    """Multiplies x and y"""
    return x * y

In [42]:
def logged(func):
    print('Adding logging to', func.__name__)
    
    def wrapper(*args, **kwargs):
        print("you called the '{}'".format(func.__name__))
        return func(*args, **kwargs)
    
    return wrapper

In [43]:
add = logged(add)
sub = logged(sub)
mul = logged(mul)

Adding logging to add
Adding logging to sub
Adding logging to mul


In [44]:
add(1, 2)

you called the 'add'


3

In [45]:
sub(20, 10)

you called the 'sub'


10

In [46]:
mul(5, 100)

you called the 'mul'


500

In [55]:
@logged
def add(x, y):
    """Adds x and y"""
    return x + y

@logged
def sub(x, y):
    """Substracts x and y"""
    return x - y

@logged
def mul(x, y):
    """Multiplies x and y"""
    return x * y

Adding logging to add
Adding logging to sub
Adding logging to mul


In [56]:
add(5, 6)

you called the 'add'


11

In [57]:
add

<function __main__.add(x, y)>

In [58]:
help(add)

Help on function add in module __main__:

add(x, y)
    Adds x and y



Metadata from original functions is lost <br>
To solve it, it's necessary:

In [59]:
from functools import wraps

def logged(func):
    print('Adding logging to', func.__name__)
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("you called the '{}'".format(func.__name__))
        return func(*args, **kwargs)
    
    return wrapper

In [61]:
@logged
def add(x, y):
    """Adds x and y"""
    return x + y

Adding logging to add


In [62]:
help(add)

Help on function add in module __main__:

add(x, y)
    Adds x and y



## Decorator with arguments

In [71]:
from functools import wraps

def logformat(message):
    def logged(func):
        print('Adding logging to', func.__name__)

        @wraps(func)
        def wrapper(*args, **kwargs):
            print(message.format(func.__name__))
            return func(*args, **kwargs)

        return wrapper
    
    return logged

In [78]:
@logformat('CALLING {}')
def add(x, y):
    """Adds x and y"""
    return x + y

Adding logging to add


In [79]:
add(1, 2)

CALLING add


3

In [82]:
@logformat('Now you are CALLING {}')
def mul(x, y):
    """Multiplies x and y"""
    return x * y

Adding logging to mul


In [83]:
mul(4, 3)

Now you are CALLING mul


12

In [85]:
decorate = logformat('Now decorating {}')
decorate

<function __main__.logformat.<locals>.logged(func)>

In [86]:
def div(x, y):
    return x / y

In [87]:
div = decorate(div)

Adding logging to div


In [88]:
div(20, 4)

Now decorating div


5.0

## Class Decorators

In [101]:
from functools import wraps

def logged(func):
    print('Adding logging to', func.__name__)
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("you called the '{}' function".format(func.__name__))
        return func(*args, **kwargs)
    
    return wrapper

In [102]:
def logmethods(cls):
    for key, val in vars(cls).items():
        if callable(val):
            setattr(cls, key, logged(val))
    return cls

In [103]:
@logmethods
class Spam:
    def __init__(self, value):
        self.value = value
        
    def yow(self):
        print('Yow', self.value)
        
    def grok(self):
        print('Grok', self.value)

Adding logging to __init__
Adding logging to yow
Adding logging to grok


In [104]:
s = Spam(2)

you called the '__init__' function


In [105]:
s.yow()

you called the 'yow' function
Yow 2


In [106]:
s.grok()

you called the 'grok' function
Grok 2


**Another Example**

In [117]:
class Typed(object):
    
    expected_type = object
    
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError("'{}' must be {}".format(self.name, self.expected_type))
        instance.__dict__[self.name] = value

class Integer(Typed):
    expected_type = int

class Float(Typed):
    expected_type = float

class String(Typed):
    expected_type = str

Class Decorator

In [119]:
def validate(**kwargs):
    def decorate(cls):
        for name, val in kwargs.items():
            setattr(cls, name, val(name))
        return cls
    return decorate

In [120]:
@validate(name=String, age=Integer, salary=Float)
class Employee:
    def __init__(self, name, age, salary):
        self.name = name
        self.age = age
        self.salary = salary

In [124]:
x = Employee('John Doe', 30, 2000.0)

In [125]:
x.name

'John Doe'

In [126]:
x.name = 10

TypeError: 'name' must be <class 'str'>

In [127]:
x.age

30

In [128]:
x.age = 30.5

TypeError: 'age' must be <class 'int'>